In [1]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)

os.environ['OPEN_API_KEY']

KeyError: 'OPEN_API_KEY'

In [17]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

embeddings = OpenAIEmbeddings(api_key=os.environ['OPEN_API_KEY'])
vector_store = Chroma(embedding_function=embeddings)
llm = ChatOpenAI(temperature=0, api_key=os.environ['OPEN_API_KEY'])

loader = WebBaseLoader(
    web_paths=['https://www.uscis.gov/working-in-the-united-states/students-and-exchange-visitors/optional-practical-training-opt-for-f-1-students'],
)
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)
vector_store.add_documents(documents=all_splits)

vector_store

In [18]:
from langchain import hub
from langchain_core.documents import Document
from typing_extensions import List, TypedDict

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    prompt = hub.pull("rlm/rag-prompt")
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

In [19]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [20]:
response = graph.invoke({'question': 'I am an international student in a STEM degree, and I want to apply for OPT. What should I know?'})
response['answer']

'As an international student in a STEM degree, you can apply for a 24-month OPT extension if you meet specific criteria. You must request your DSO to recommend OPT and file Form I-765 with USCIS. The application timing varies based on whether you are applying for pre-completion or post-completion OPT.'

LangGraph agent